In [1]:
pip install cpe

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests 
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from cpe import CPE
import time

In [5]:
# Create SQLite database
conn = sqlite3.connect('nvd_cve.db')

# Create a cursor object
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE cve (
    id INTEGER PRIMARY KEY,
    cve VARCHAR(20) NOT NULL,
    published TIMESTAMP NOT NULL,
    last_modified TIMESTAMP NOT NULL,
    vuln_status VARCHAR(8) NOT NULL CHECK (vuln_status IN ('ANALYZED','MODIFIED')),
    cvss_ver INTEGER NOT NULL CHECK (cvss_ver IN (2, 3)),
    base_severity VARCHAR(8) NOT NULL CHECK (base_severity IN ('LOW', 'MEDIUM', 'HIGH', 'CRITICAL')),
    base_score REAL NOT NULL CHECK (base_score >= 0.0 AND base_score <= 10.0),
    impact_score REAL NOT NULL CHECK (impact_score >= 0.0 AND impact_score <= 10.0),
    exploitability_score REAL NOT NULL CHECK (exploitability_score >= 0.0 AND exploitability_score <= 10.0),
    vector_string_v2 TEXT,
    vector_string_v3 TEXT,
    last_ingested TIMESTAMP NOT NULL
);
''')

cursor.execute('''
CREATE TABLE cve_product (
    metric_id INTEGER PRIMARY KEY AUTOINCREMENT,
    cve_id INTEGER NOT NULL,
    operator VARCHAR(3) NOT NULL CHECK (operator IN ('OR', 'AND')),
    negate BOOLEAN,
    cpe_name TEXT NOT NULL,
    match_criteria_id TEXT NOT NULL,
    vulnerable BOOLEAN,
    vendor TEXT NOT NULL,
    product TEXT NOT NULL,
    last_ingested TIMESTAMP NOT NULL  
);
''')

conn.commit()

In [6]:
def add_data_db(df_raw,db_table):
    df = pd.DataFrame(df_raw)

    df['last_ingested'] = datetime.now()

    df.to_sql(db_table, conn, if_exists='append', index=False)

    df_json = None
    df = None
    

In [ ]:
base_url = 'https://services.nvd.nist.gov/rest/json/cves/2.0'

noRejected = 'noRejected'

target_date = '2024-05-01 23:59:59.599'
target_date = datetime.strptime(target_date, '%Y-%m-%d %H:%M:%S.%f')

is_target_found = 0

startIndex = 0

resultsPerPage = 2000 #max allowed

missing_data = []

start_time = datetime.now()

while is_target_found == 0:

    incremental_start_time = datetime.now()

    print(startIndex,startIndex + resultsPerPage)

    #collect data

    query_url = base_url + '/?' + f'resultsPerPage={resultsPerPage}' + '&' + f'startIndex={startIndex}' \
                         + '&' + noRejected 

    request_successful = False

    try_n = 0

    #got data or not?

    while not request_successful and try_n < 10:

        try:
            response = requests.get(query_url)

            if response.status_code == 200:
                request_successful = True

            else:
                try_n += 1
                #delay
                time.sleep(5)

        except Exception as e:
            try_n += 1
            #delay
            time.sleep(5)

    if not request_successful:
        print(f'missing data, start_index = {startIndex}')
        missing_data += [startIndex]
        startIndex += resultsPerPage
        continue

    data = response.json()

    data = data['vulnerabilities']

    #check target
    max_date = max(data, key=lambda x: datetime.strptime(x['cve']['published'], '%Y-%m-%dT%H:%M:%S.%f'))['cve']['published']
    max_date = datetime.strptime(max_date, '%Y-%m-%dT%H:%M:%S.%f')

    #Pre filter data
    data = [cve for cve in data if (datetime.strptime(cve['cve']['published'], '%Y-%m-%dT%H:%M:%S.%f') <= target_date) &\
                                    (datetime.strptime(cve['cve']['lastModified'], '%Y-%m-%dT%H:%M:%S.%f') <= target_date) &\
                                    (cve['cve']['vulnStatus'].upper() in ['ANALYZED','MODIFIED']) &\
                                    (any(element in ['cvssMetricV2','cvssMetricV30'] for element in cve['cve']['metrics'].keys()))
            ]
    
#https://www.first.org/cvss/v3.0/user-guide

    cve = [
                {   'id' : index + startIndex,
                    'cve' : i['cve']['id'],
                    'published' : i['cve']['published'],
                    'last_modified' : i['cve']['lastModified'],
                    'vuln_status' : i['cve']['vulnStatus'].upper(),
                    'cvss_ver' : 3 if 'cvssMetricV30' in i['cve']['metrics'] else 2,
                    'base_severity' : i['cve']['metrics']['cvssMetricV30'][0]['cvssData']['baseSeverity'] if 'cvssMetricV30' in i['cve']['metrics'] \
                                    else i['cve']['metrics']['cvssMetricV2'][0]['baseSeverity'],
                    'base_score' : i['cve']['metrics']['cvssMetricV30'][0]['cvssData']['baseScore'] if 'cvssMetricV30' in i['cve']['metrics'] \
                                else i['cve']['metrics']['cvssMetricV2'][0]['cvssData']['baseScore'],
                    'impact_score': i['cve']['metrics']['cvssMetricV30'][0]['impactScore'] if 'cvssMetricV30' in i['cve']['metrics'] \
                                    else i['cve']['metrics']['cvssMetricV2'][0]['impactScore'],
                    'exploitability_score' : i['cve']['metrics']['cvssMetricV30'][0]['exploitabilityScore'] if 'cvssMetricV30' in i['cve']['metrics'] \
                                            else i['cve']['metrics']['cvssMetricV2'][0]['exploitabilityScore'],
                    'vector_string_v2' : i['cve']['metrics']['cvssMetricV2'][0]['cvssData']['vectorString'] if 'cvssMetricV2' in i['cve']['metrics'] \
                                    else np.nan,
                    'vector_string_v3' : i['cve']['metrics']['cvssMetricV30'][0]['cvssData']['vectorString'] if 'cvssMetricV30' in i['cve']['metrics'] \
                                    else np.nan
                } 
                for index,i in enumerate(data)
           ]

    add_data_db(cve,'cve')
    
    cve_products = [
                        {
                            'cve_id': index + startIndex,
                            'operator' : node['operator'].upper(),
                            'negate' : node['negate'],
                            'cpe_name' : cpe['criteria'],
                            'match_criteria_id' : cpe['matchCriteriaId'],
                            'vulnerable' : cpe['vulnerable'],
                            'vendor' : CPE(cpe['criteria']).get_vendor()[0],
                            'product' : CPE(cpe['criteria']).get_product()[0],
                        }
                        for index,i in enumerate(data)
                        for node in i['cve'].get('configurations', [{}])[0].get('nodes', [])
                        for cpe in node.get('cpeMatch', [])
                    ]




    add_data_db(cve_products,'cve_product')

    incremental_end_time = datetime.now()

    print(incremental_end_time - incremental_start_time)
    
    if max_date > target_date:

        is_target_found = 1

        print(f'Target ( {target_date} ) breached, stop collecting')

        break

    #collect more data
    startIndex += resultsPerPage


end_time = datetime.now()

print(f'Total time: {end_time - start_time}')

if len(missing_data) > 0:
    print(f'missing incrementals: {missing_data}') 




0 2000
0:00:04.209101
2000 4000
0:00:04.637391
4000 6000
0:00:05.415119
6000 8000
0:00:06.257560
8000 10000
0:00:07.536619
10000 12000
0:00:29.812251
12000 14000
0:00:06.941300
14000 16000
0:00:06.721936
16000 18000
0:00:06.428791
18000 20000
0:00:06.550678
20000 22000
0:00:05.689831
22000 24000
0:00:06.144035
24000 26000
0:00:06.219314
26000 28000
0:00:06.414993
28000 30000
0:00:07.415068
30000 32000
0:00:08.678387
32000 34000
0:00:09.735334
34000 36000
0:00:09.838178
36000 38000
0:00:12.009512
38000 40000
0:01:36.271632
40000 42000
0:00:11.005875
42000 44000
0:00:11.298591
44000 46000
0:00:16.174145
46000 48000
0:00:14.533839
48000 50000
0:00:14.146972
50000 52000
0:00:14.406259
52000 54000
0:00:13.282318
54000 56000
0:01:44.867935
56000 58000
0:00:13.364703
58000 60000
0:00:13.457780
60000 62000
0:01:32.092813
62000 64000
0:00:07.572275
64000 66000
0:00:06.821082
66000 68000
0:00:06.994502
68000 70000
0:00:07.902318
70000 72000
0:00:06.349406
72000 74000
0:00:06.660497
74000 76000
0